In [ ]:
import pandas as pd
import numpy as np

# Load data files into dataframes with separator "|" or ","
bnc = pd.read_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/nsc2_edu_bnc.txt", sep="|")
bnd = pd.read_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/nsc2_edu_bnd.txt", sep="|")
g1e = pd.read_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/nsc2_edu_g1e.txt", sep="|", encoding='cp949')
m20 = pd.read_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/nsc2_edu_m20.txt", sep=",")

# Correct birth year in bnd and convert to numeric
bnd['BTH_YYYY'] = bnd['BTH_YYYY'].replace('1921LE', '1921').astype(int)
# Add randomness to birth year
np.random.seed(1234)
bnd['BTH_YYYY'] = bnd['BTH_YYYY'] + np.random.choice(range(10), size=len(bnd))


In [13]:
### 1. 2006년 건강검진을 받은 사람들을 추출
bnc_2006 = bnc[bnc['STD_YYYY'] == 2006]
g1e_2006_2012 = g1e[['RN_INDI', 'G1E_BMI', 'G1E_BP_SYS', 'G1E_TOT_CHOL', 'G1E_FBS', 'G1E_HGB', 'Q_FHX_HTDZ', 'Q_FHX_STK']]
print(f"Number of people in 2006: {len(bnc_2006)}")
print(f"Number of people who received health checkups in 2006: {len(g1e_2006_2012)}")
#Number of people in 2006: 3000
#Number of people who received health checkups in 2006: 2496

Number of people in 2006: 3000
Number of people who received health checkups in 2006: 2496


In [18]:
### 2. 관심 변수 중 Missing value가 있는 사람들 제외
coh_people = g1e_2006_2012.dropna()
print(f"Number of rows without missing values: {len(coh_people)}")
#Number of rows without missing values: 1938
print(f"Number of people without missing values: {len(coh_people['RN_INDI'].unique())}")
#Number of people without missing values: 1019

# Note: g1e_2006_2012 테이블에는 한 사람당 검진 결과가 여러 번 있을 수 있다. 따라서 검진 결과를 summarise하여 한 사람당 한 행으로 만든다.
# RN_INDI를 기준으로 관심 변수를 summarise
# G1E_BMI -> mean
# G1E_BP_SYS -> mean
# G1E_TOT_CHOL -> mean
# G1E_FBS -> mean
# G1E_HGB -> mean
# Q_FHX_HTDZ -> max
# Q_FHX_STK -> max
coh_people = coh_people.groupby('RN_INDI').agg({
    'G1E_BMI': 'mean',
    'G1E_BP_SYS': 'mean',
    'G1E_TOT_CHOL': 'mean',
    'G1E_FBS': 'mean',
    'G1E_HGB': 'mean',
    'Q_FHX_HTDZ': 'max',
    'Q_FHX_STK': 'max'
}).reset_index()
print(f"Number of people without missing values after summarising: {len(coh_people)}")
#Number of people without missing values after summarising: 1019

# 심혈관질환 가족력 (FHX_CVD) 변수 생성 (Q_FHX_HTDZ == 2 or Q_FHX_STK == 2)
coh_people['FHX_CVD'] = np.where((coh_people['Q_FHX_HTDZ'] == 2) | (coh_people['Q_FHX_STK'] == 2), 1, 0)
# Q_FHX_HTDZ, Q_FHX_STK 변수 제거
coh_people = coh_people.drop(columns=['Q_FHX_HTDZ', 'Q_FHX_STK'])

# bnc_2006 데이터 (SEX)와 bnd 데이터 (BTH_YYYY)를 coh_people에 left_join
coh_people = pd.merge(coh_people, bnc_2006[['RN_INDI', 'SEX']], on='RN_INDI', how='left')
coh_people = pd.merge(coh_people, bnd[['RN_INDI', 'BTH_YYYY']], on='RN_INDI', how='left')
print(f"Number of people after joining: {len(coh_people)}")
#Number of people after joining: 1019
# 나이 (AGE) 변수 생성
coh_people['AGE'] = 2006 - coh_people['BTH_YYYY']

Number of rows without missing values: 1938
Number of people without missing values: 1019
Number of people without missing values after summarising: 1019
Number of people after joining: 1019


In [19]:
# m20 데이터에서 고혈압 진단 코드 (I10)를 가진 사람들을 추출
# event_code는 I10
event_code = ['I10']
# coh_people에 있는 사람들만 m20 데이터에서 추출
m20_2 = m20[m20['RN_INDI'].isin(coh_people['RN_INDI'])]
# m20_2의 SICK_SYM1 또는 SICK_SYM2 에서 event_code를 포함하는 사람들을 추출
m20_event = m20_2[m20_2[['SICK_SYM1', 'SICK_SYM2']].apply(lambda x: x.str.contains('|'.join(event_code))).any(axis=1)]
# RN_INDI를 기준으로 가장 빠른 고혈압 진단일을 선택
m20_event = m20_event.groupby('RN_INDI').apply(lambda x: x.loc[x['MDCARE_STRT_DT'].idxmin()]).reset_index(drop=True)
# HYP 변수 생성
m20_event['HYP'] = 1

# m20 데이터에서 ASCVD 진단 코드를 가진 사람들을 추출
# event2_code는 I20~I25, G45, G46, I63, I64, I65, I66, I672, I694, I70, I738, I739
event2_code = ['I' + str(i) for i in range(20, 26)] + ['G45', 'G46', 'I63', 'I64', 'I65', 'I66', 'I672', 'I694', 'I70', 'I738', 'I739']
# m20_2의 SICK_SYM1 또는 SICK_SYM2 에서 event2_code를 포함하는 사람들을 추출
m20_event2 = m20_2[m20_2[['SICK_SYM1', 'SICK_SYM2']].apply(lambda x: x.str.contains('|'.join(event2_code))).any(axis=1)]
# RN_INDI를 기준으로 가장 빠른 ASCVD 진단일을 선택
m20_event2 = m20_event2.groupby('RN_INDI').apply(lambda x: x.loc[x['MDCARE_STRT_DT'].idxmin()]).reset_index(drop=True)
# ASCVD 변수 생성
m20_event2['ASCVD'] = 1

# coh_people에 HYP, MDCARE_STRT_DT(rename - HYP_DATE) left_join하고 
coh_people = pd.merge(coh_people, m20_event[['RN_INDI', 'HYP', 'MDCARE_STRT_DT']].rename(columns={'MDCARE_STRT_DT': 'HYP_DATE'}), on='RN_INDI', how='left')
# HYP 변수의 missing value를 0으로 채움
coh_people['HYP'] = coh_people['HYP'].fillna(0)
# coh_people에 ASCVD, MDCARE_STRT_DT(rename - ASCVD_DATE) left_join 
coh_people = pd.merge(coh_people, m20_event2[['RN_INDI', 'ASCVD', 'MDCARE_STRT_DT']].rename(columns={'MDCARE_STRT_DT': 'ASCVD_DATE'}), on='RN_INDI', how='left')
# ASCVD 변수의 missing value를 0으로 채움
coh_people['ASCVD'] = coh_people['ASCVD'].fillna(0)

# 3. 최종 인원 선택
print(f"Number of people after merging HYP and ASCVD events: {len(coh_people)}")

# Add some randomness to RN_INDI for privacy protection
np.random.seed(1234)
coh_people['RN_INDI'] = coh_people['RN_INDI'] + np.random.choice(range(1, 101), size=len(coh_people))
# Select and reorder columns
coh_people = coh_people[['RN_INDI', 'SEX', 'BTH_YYYY', 'AGE', 'G1E_BMI', 'G1E_BP_SYS', 'G1E_FBS', 'G1E_HGB', 'G1E_TOT_CHOL', 'FHX_CVD', 'HYP', 'HYP_DATE', 'ASCVD', 'ASCVD_DATE']]
# Save the cohort data to a CSV file
coh_people.to_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/hyp_ascvd_cohort.csv", index=False)

Number of people after merging HYP and ASCVD events: 1019


/tmp/ipykernel_47448/2694877838.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  m20_event = m20_event.groupby('RN_INDI').apply(lambda x: x.loc[x['MDCARE_STRT_DT'].idxmin()]).reset_index(drop=True)
/tmp/ipykernel_47448/2694877838.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  m20_event2 = m20_event2.groupby('RN_INDI').apply(lambda x: x.loc[x['MDCARE_STRT_DT'].idxmin()]).reset_index(drop=True)


In [21]:
# 4. train, val, test로 구분 (8:1:1)
num_train = int(len(coh_people) * 0.8)
num_val = int(len(coh_people) * 0.1)
num_test = len(coh_people) - num_train - num_val

# randomly select indices for train, val, test
np.random.seed(1234)
indices = np.random.permutation(len(coh_people))
train_indices = indices[:num_train]
val_indices = indices[num_train:num_train+num_val]
test_indices = indices[num_train+num_val:]

# split the dataset
train = coh_people.iloc[train_indices].reset_index(drop=True)
val = coh_people.iloc[val_indices].reset_index(drop=True)
test = coh_people.iloc[test_indices].reset_index(drop=True)

# print number of rows
print(f"Number of rows in train: {len(train)}")
print(f"Number of rows in val: {len(val)}")
print(f"Number of rows in test: {len(test)}")
#Number of rows in train: 815
#Number of rows in val: 101
#Number of rows in test: 103

# save the dataset
train.to_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/hyp_ascvd_cohort_train.csv", index=False)
val.to_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/hyp_ascvd_cohort_val.csv", index=False)
test.to_csv("/mnt/c/AI_practice_20250314/data/nhis_edu/hyp_ascvd_cohort_test.csv", index=False)

Number of rows in train: 815
Number of rows in val: 101
Number of rows in test: 103
